<a href="https://colab.research.google.com/github/sujaykar/Echo/blob/main/ChatGPT_with_your_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Requirement

In [ ]:
# You might need to reinstall Pillow library if you receive PIL error
# !pip uninstall Pillow
# !pip install --upgrade Pillow
# import PIL
# print(PIL.__version__)

In [ ]:
!pip install openai -q
!pip install langchain -q
!pip install chromadb -q
!pip install tiktoken -q
!pip install pypdf -q
!pip install unstructured[local-inference] -q
!pip install gradio -q

## Import Libraries

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain

import os
os.environ["OPENAI_API_KEY"] = "Your OpenAI Key Here"

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,model_name="gpt-4")

# Load your data into Reports folder

In [ ]:
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('./Reports/', glob="**/*.pdf")
txt_loader = DirectoryLoader('./Reports/', glob="**/*.txt")
word_loader = DirectoryLoader('./Reports/', glob="**/*.docx")

loaders = [pdf_loader, txt_loader, word_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")


Total number of documents: 3


# Chunk the data, turn into Embeddings and save to VectorStore

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

# Calling the Langchain's QA chain with Chat History

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

# Gradio Chat UI

In [ ]:
import gradio as gr
# Define chat interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []

    def user(query, chat_history):
        print("User query:", query)
        print("Chat history:", chat_history)

        # Convert chat history to list of tuples
        chat_history_tuples = []
        for message in chat_history:
            chat_history_tuples.append((message[0], message[1]))

        # Get result from QA chain
        result = qa({"question": query, "chat_history": chat_history_tuples})

        # Append user message and response to chat history
        chat_history.append((query, result["answer"]))
        print("Updated chat history:", chat_history)

        return gr.update(value=""), chat_history


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

User query: You were given three documents as context
Chat history: []
Updated chat history: [('You were given three documents as context', 'Yes, that is correct. The first document appears to be a speech about the state of the union, the second document seems to be a list of updates and features for various Microsoft products, and the third document mentions a change in estimated useful lives of servers and network equipment. Is there anything specific you would like me to help you with?')]
User query: Summarize each documents for me
Chat history: [['You were given three documents as context', 'Yes, that is correct. The first document appears to be a speech about the state of the union, the second document seems to be a list of updates and features for various Microsoft products, and the third document mentions a change in estimated useful lives of servers and network equipment. Is there anything specific you would like me to help you with?']]
Updated chat history: [['You were given t